In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
%pip install -qq -U datasets transformers pyarrow
%pip install -qq --upgrade transformers ftfy accelerate regex tqdm
%pip install git+https://github.com/openai/CLIP.git

Let's import all the libraries we need

In [9]:
from tqdm.auto import tqdm
import clip
import os
import torch

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"


**Download the CLIP model to encode the image**

In [12]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 141MiB/s]


**Let's Write a method to encode the image using clip model**

In [14]:
def calc_image_emb(img, model, preprocess, device):
    """
    This method computes the clip embeddings for a given image, after preprocessing it according to the model
    """
    image = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features
    


In [17]:
import requests
from PIL import Image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
img_features = calc_image_emb(image, clip_model, clip_preprocess, device)
print(img_features)

In [20]:
img_features.shape

torch.Size([1, 512])

In [23]:
from datasets import load_dataset

dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K",split='train')

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [24]:
!curl -X GET \
     "https://datasets-server.huggingface.co/splits?dataset=liuhaotian%2FLLaVA-Instruct-150K"